The idea is to put only important algorithms to Review

# DIFFERENCE ARRAY (LEET 2381)

Da usare quando si vuole fare un range update, invece che aggiornare ogni elemento da index 2 a 5 ad esempio. Incremento l’elemento all’indice 2 di + tot e tolgo all’indice end +1 – tot quindi se voglio aggiungere 2 a tutti gli elementi tra 2 e 5 devo fare

Start = [1, 2, 1, 1, 3, 0, 0]

Compute gradient array: array delle somme parziali, cioè prendo solo le differenze tra elementi

Arra_diff= [1, 1, -1, 0, 2, -3, 0, 0]   

last element is dummy. Il dummy serve se voglio aggiornare un range che ha dentro anche l’ultimo index dell’array

Aggiungo il valore a index 2 e tolgo a index 5+1:

Arra_diff= [1, 1, 1, 0, 2, -5, 0, 0]

Poi per ottenere la somma finale devo fare la somma cumulativa, l’ultimo elemento dell’array sum è dummy

Tot_sum = [1, 2, 3, 3, 5, 0,  0, 0]

Final_res = [1, 2, 3, 3, 5, 0,  0] #remove last element

In [ ]:
class Solution:
    def shiftingLetters(self, s: str, shifts: List[List[int]]) -> str:
        #0 sta dov'è, -num vai indietro e +num vai inavanti
        arr_diff = [0] * (len(s)+1)
        final_shifts = []
        for shift in shifts:
            if shift[2]:
                arr_diff[shift[0]] += 1
                arr_diff[shift[1]+1] -= 1
            else:
                arr_diff[shift[0]] -= 1
                arr_diff[shift[1]+1] += 1

        shift = 0
        for i in range(len(arr_diff)-1):
            shift += arr_diff[i]
            final_shifts.append(shift)

        res = ""
        for i, el in enumerate(final_shifts):
            new_char = chr((ord(s[i]) - ord('a') + el) % 26 + ord('a'))
            res += new_char
        return res


 # BIT (LEET 307) MUTABLE QUERY RANGE
 It is possible to use also segment tree tree

https://www.youtube.com/watch?v=uSFzHCZ4E-8 (BIT explanation)

 https://cp-algorithms.com/data_structures/fenwick.html   (binary)

 https://cp-algorithms.com/data_structures/segment_tree.html  (segment)

 We need tree structure because there is an updated otherwise a sum array would be suffienct e.g.:
 start = [ 5, 2, 9, -3, 5]

 summ array = [5, 7, 16, 13, 18]

 sum from index 1 to 3: summ_array[3] - sum_array[0] = 13-5 = 8

 So basically to get the sum from i to j, you need to take arr[j] - arr[i-1]
 but it doesnt work with updates (update in O(N) time) so we need to use a tree structure.

-------------------------------------------------------------------
BIT

 The magic of index & -index lies in efficiently isolating and leveraging the lowest set bit:

Addition (+=): Move to the next index that overlaps the current range.
Subtraction (-=): Move to the parent index, narrowing the range.

use addition for updates and subtraction for summing, to get the parent node

Intuition Behind the Difference
Updates (+= index & -index): Propagate changes forward to larger indices that depend on the current index.
Queries (-= index & -index): Aggregate contributions backward from smaller indices that form the prefix sum.







In [ ]:
class BIT:
      """
    Implementation of Binary Indexed Tree/Fenwick Tree

    Memory:
        creation - O(n)
        update   - O(1)
        get_sum  - O(1)

    Time:
        creation - O(n*log(n))
        update   - O(log(n))
        get_sum  - O(log(n))
    """
    def __init__(self, nums: List[int]):
        self.size = len(nums)
        self.bit = [0] * (self.size +1)
        for i in range(self.size):
            self.update(i+1, nums[i])

    def update(self, index: int, delta: int) -> None:
        while index <= self.size:
            self.bit[index] += delta
            index += index & -index #set LSB

    def query(self, index) -> int:
        res = 0
        while index > 0:
            res += self.bit[index]
            index -= index & -index
        return res

class NumArray:
    """
    Time:   O(N∗Log(N))
    Memory: O(n)
    """
    def __init__(self, nums: List[int]):
        self.nums = nums
        self.bit = BIT(nums)

    def update(self, index: int, val: int) -> None:
        self.bit.update(index + 1, val - self.nums[index])
        self.nums[index] = val

    def sumRange(self, left: int, right: int) -> int:
        return self.bit.query(right+1) - self.bit.query(left)


# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# obj.update(index,val)
# param_2 = obj.sumRange(left,right)

# IMMUTABLE QUERY RANGE (LEET 303)

non serve la tree structure perchè non abbiamo updates
quindi basta utilizzare l'array della prefix sum, cioè somma cumulativa e fare somm:  arr[right] - arr[left-1]

In [ ]:
class NumArray:

    def __init__(self, nums: List[int]):
        self.sum = [0] * (len(nums)+1)

        #somma cumulativa
        for i in range(len(nums)):
            self.sum[i+1] = self.sum[i] + nums[i]

    #ho spostato gli indici perchè ho fatto la sum di len n +1 così risulta + facile fare left-1, quindi left-1 +1 = left e right +1 = right+1
    #così posso riferirmi all'array di sum facilmente
    def sumRange(self, left: int, right: int) -> int:
        return self.sum[right+1] - self.sum[left]


# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# param_1 = obj.sumRange(left,right)

# PREFIX SUM & SUFFIX SUM (LEET 1769)

In questo caso farlo senza prefix e suffix sarebbe n^2. Quello che faccio con prexis sum e suffix sum è calcolarsi prima la somma da sx a dx e poi da dx a sx, il risultato finale è dato dalla somma di prefix sum e suffix sum.

In questo caso nel leet si poteva ottenere prefix sum e suffix sum tenendosi il numero di biglie incontrate fino a quel punto e aggiornandolo in caso si trovasse una nuova biglia, e lo si aggiungeva al risultato corrente in modo che il numero totale di biglie da muovere venisse aggiornato e quindi anche il numero totale di movimenti.

In [ ]:
class Solution:
    def minOperations(self, boxes: str) -> List[int]:
        n = len(boxes)
        #computo da sx a dx e da dx a sx così posso farlo in tempo n,
        #ogni volta aggiungo il numero di palle perchè per andare da una posizione ad una adiacente mi basta
        #computare il numero di palle e aggiungerle perchè devo spostare ogni volta tot palle per inserle nel box i
        l_r = [0] * n
        r_l = [0] * n

        tot_balls = 0
        tot_operations = 0
        for i in range(n):
            tot_operations += tot_balls
            l_r[i] = tot_operations
            if boxes[i] == '1':
                tot_balls += 1

        tot_balls = 0
        tot_operations = 0
        for i in reversed(range(n)):
            tot_operations += tot_balls
            r_l[i] = tot_operations
            if boxes[i] == '1':
                tot_balls += 1

        for i in range(n):
            l_r[i] += r_l[i]
        return l_r

GRAPH SHORTEST PATH (LEET 1368)

if weights are 0-1 in a directed graph you can use BFS 0-1 that means if weight is 1 append it to the right if it 0 have priority so append it to left.
Otherwise use heapqueue

In [ ]:
class Solution:
    def minCost(self, grid: List[List[int]]) -> int:
        directions = [(0,1,1), (0,-1,2), (1,0,3), (-1,0,4)]
        queue = deque([(0,0,0)]) # row, col , path
        visited = set()
        rows = len(grid)
        cols = len(grid[0])

        while queue:
            r, c, path = queue.popleft()
            if (r,c) == (rows-1, cols-1):
                return path
            if (r,c) in visited:
                continue
            visited.add((r,c))

            for x,y,d in directions:
                if 0 <= x + r < rows and 0 <= y + c < cols and (r+x, c+y):
                    if d == grid[r][c]:
                        queue.appendleft((r+x, c+y, path))
                    else:
                        queue.append((r+x, c+y, path + 1))